In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from functools import *
from selenium import webdriver
import time
from datetime import datetime

In [50]:
original_csv = 'combined_articles.csv'

home_url = 'https://hjp.znanje.hr/'
search_url = 'https://hjp.znanje.hr/index.php?show=search'

noun_tags = ['m', 'ž', 'sr']
verb_tags = ['svrš.', 'nesvrš.', 'dv.']


multiple_entries_found_string = 'Pronađeno je više natuknica koje odgovaraju zadanom uvjetu:'
no_entries_found_string = 'Nijedna natuknica u rječničkoj bazi ne zadovoljava zadane uvjete'
extract_prefix = True

allowed_types = [noun_tags]
allowed_types = reduce(lambda a, b: a+b, allowed_types)

In [51]:
def read_dict():
    df = pd.read_csv('dictionary_index_articles.csv').fillna('')
    return df.to_dict()
    
def save_dict(d):
    k = np.array(list(d.keys()))
    v = np.array(list(d.values()))
    df = pd.DataFrame(np.stack([k, v], axis=-1))
    df.to_csv('dictionary_' + original_csv, index=False)

try:
    dictionary = read_dict()
except FileNotFoundError:
    dictionary = {'': ''}

In [60]:
def extract_prefix(word, do_extract):
    if not do_extract:
        return word
    
    l = len(word)
    if l > 10:
        return word[:8]
    elif l > 7:
        return word[:5]
    elif l > 5:
        return word[:3]
    elif l > 3:
        return word[:2]
    return word
    
    
def _normalize_word_req(word, extract_prefix):
    word_page = requests.post(search_url, data={'word': word}).content
    if multiple_entries_found_string in word_page.decode("utf-8"):
        soup = BeautifulSoup(word_page)
        element = soup.find('div', class_= 'container content grey')
        link = home_url + element.find('a')['href']
        word_page = requests.get(link).content
    elif no_entries_found_string in word_page.decode("utf-8"):
        return extract_prefix(word, extract_prefix)
    try:
        soup = BeautifulSoup(word_page)
        element = soup.find('div', class_= 'container content grey').find('div').find('div')
        header = element.find_all('p')[1]
        word_type = header.find('i').text
        if word_type not in allowed_types:
            word_normal = ''
        else:
            word_normal = header.find('b').text
    except (AttributeError, IndexError):
        word_normal = ''
        word_type = ''
        
    return word_normal


def normalize_word(word, extract_prefix=True):
    if word in dictionary:
        return dictionary.get(word)
    
    normalized = _normalize_word_req(word, extract_prefix)
    dictionary[word] = normalized
    return normalized


def filter_and_normalize(words, extract_prefix=True):
    normalized = list(map(lambda word: normalize_word(word, extract_prefix), words))
    filtered = list(filter(lambda x: x != '', normalized))
    return filtered

def normalize_text(text):
    try:
        words = re.split('[<>()\[\]{}|\\/-_,.;:!?\"\' \n\t]', text)
        normalized = filter_and_normalize(words, extract_prefix)
#     print(normalized)
    
        return reduce(lambda a, b: a + ' ' + b, normalized)
    except TypeError:
#         print(normalized)
        return ''

In [61]:
database = pd.read_csv(original_csv)
texts = database['content']
mod_texts = [[] for _ in texts]
for i, text in enumerate(texts):
    start = time.time()
    mod_texts[i] = normalize_text(text)
    end = time.time()
    print("Time {}: {}".format(i, end - start))
    save_dict(dictionary)
# modified_database.to_csv('modified_' + original_csv, index=False)
database['modified'] = mod_texts
database.to_csv('modified.csv', index = False)

Time 0: 0.0009958744049072266
Time 1: 0.001994609832763672
Time 2: 0.001994609832763672
Time 3: 0.00199127197265625
Time 4: 0.0019922256469726562
Time 5: 0.0019927024841308594
Time 6: 0.002004384994506836
Time 7: 0.0010099411010742188
Time 8: 0.000997304916381836
Time 9: 0.0010006427764892578
Time 10: 0.0020318031311035156
Time 11: 0.0019931793212890625
Time 12: 0.001994609832763672
Time 13: 0.0009906291961669922
Time 14: 0.000995635986328125
Time 15: 0.001995086669921875
Time 16: 0.000997304916381836
Time 17: 0.0
Time 18: 0.000997304916381836
Time 19: 0.003987789154052734
Time 20: 0.0019898414611816406
Time 21: 0.005982160568237305
Time 22: 0.0029926300048828125
Time 23: 0.0009970664978027344
Time 24: 0.0009982585906982422
Time 25: 0.0009961128234863281
Time 26: 0.0009982585906982422
Time 27: 0.0009965896606445312
Time 28: 0.0009872913360595703
Time 29: 0.0
Time 30: 0.0009958744049072266
Time 31: 0.0
Time 32: 0.000997304916381836
Time 33: 0.001046895980834961
Time 34: 0.00099992752075

In [55]:
for i, text in enumerate(texts[170:]):
    i += 170
    start = time.time()
    mod_texts[i] = normalize_text(text)
    end = time.time()
    print("Time {}: {}".format(i, end - start))
    save_dict(dictionary)
# modified_database.to_csv('modified_' + original_csv, index=False)
database['modified'] = mod_texts
database.to_csv('modified.csv', index = False)

Time 170: 35.18050289154053
Time 171: 11.49396562576294


TypeError: expected string or bytes-like object

In [43]:
for i, text in enumerate(texts[:5]):
    if mod_texts[i] != '':
        print(mod_texts[i])
        print(i)

kompànija dȍstava hrána oor ash olt poslovánje ur rázdōblje normalizácija potrážnja pandemij bu otraž ȕsluga hrána pandèmija cov mjȅsec priprema konsolidácija sȅktor eut ransa milìjārda èuro mer oor kúpnja ol transàkcija milìjārda èuro nȕžda diòničār Fȋnskā tvȑtka diònīk rijed oor ash diònica sklȍp transàkcija dȍlār Pȃd diònīk sȕ dvȉje kompànija oor ash transàkcija tȑžīšte an Švȅdskā rva rb olt gȍdina elsin međuvrijéme ur zi tìsuća pȁrtner restòrān tvȑtka návod eut olt im kòrisnīk in kompànija krȃj kòrisnīk oor ash krȃj tròmjesēčje prètplatnīk ȕsluga druži snága ol nȃm ràzina fókus priopćénje objavlje útorak šȅf1 tvȑtka ony ot dvȉje trećìna vrijédnōst olt nárudžba pokàzatelj koja ukl vrijédnōst nárudžba pretplat ȕsluga dȍprinos sȕ tvȑtka dètālj     gȍdina zȃrada dȍlār ransa polòvina gȍdina ol oor as prilagođ ráspon nȕla dȍlār oor ash príhod tròmjesēčje milìjārda dȍlār
0
vrhúnac karijéra svijȇt ténis svo brȏnca limpi ìgra av ćȕp gȍdina ìgrāč svijȇt karijéra bȍlēst rȁzgovōr -o emìsija fi

In [45]:
mod_texts

['kompànija dȍstava hrána oor ash olt poslovánje ur rázdōblje normalizácija potrážnja pandemij bu otraž ȕsluga hrána pandèmija cov mjȅsec priprema konsolidácija sȅktor eut ransa milìjārda èuro mer oor kúpnja ol transàkcija milìjārda èuro nȕžda diòničār Fȋnskā tvȑtka diònīk rijed oor ash diònica sklȍp transàkcija dȍlār Pȃd diònīk sȕ dvȉje kompànija oor ash transàkcija tȑžīšte an Švȅdskā rva rb olt gȍdina elsin međuvrijéme ur zi tìsuća pȁrtner restòrān tvȑtka návod eut olt im kòrisnīk in kompànija krȃj kòrisnīk oor ash krȃj tròmjesēčje prètplatnīk ȕsluga druži snága ol nȃm ràzina fókus priopćénje objavlje útorak šȅf1 tvȑtka ony ot dvȉje trećìna vrijédnōst olt nárudžba pokàzatelj koja\xa0ukl vrijédnōst nárudžba pretplat ȕsluga dȍprinos sȕ tvȑtka dètālj \xa0\xa0\xa0 gȍdina zȃrada dȍlār ransa polòvina gȍdina ol oor as prilagođ ráspon nȕla dȍlār oor ash príhod tròmjesēčje milìjārda dȍlār',
 'vrhúnac karijéra svijȇt ténis svo brȏnca limpi ìgra av ćȕp gȍdina ìgrāč svijȇt karijéra bȍlēst rȁzgov

In [46]:
database['modified'] = mod_texts

In [56]:
database['modified'] = mod_texts
database.to_csv('modified.csv', index = False)